# 🗞️ OLJ Customer Segmentation Heuristics 8-Week Rolling Data Analysis Script




<img src="https://img.lemde.fr/2024/03/20/354/0/4064/2032/1342/671/60/0/8672e76_1710973858299-000-nic6404406.jpg" alt="Bike Store" width="200" align="right">

This is the Engagement axis calculation for our users.

It relies on 2 Matomo reports..
1. User Frequency Report to get Depth + Category Splits
2. User Cadency Report to get Cadency dimension, but still factors in WeeksActive from User Frequency Report

Step 1: Make an Engagement Data Frame

In [ ]:
rm -rf /content/* /content/.[!.]* /content/..?*


In [ ]:
# === Clean User Engagement Report (exact columns + pretty durations) → save in /content/cleaned ===
import os
import re
import math
import pandas as pd
from google.colab import files
from datetime import date
from IPython.display import display

# ---------- Safe base and output directory ----------
BASE_DIR = "/content" if os.path.isdir("/content") else os.getcwd()
CLEAN_DIR = os.path.join(BASE_DIR, "cleaned")
os.makedirs(CLEAN_DIR, exist_ok=True)
print("Saving outputs to:", CLEAN_DIR)

# ---------- Upload files ----------
uploaded = files.upload()   # multi-select ok
names = sorted(uploaded.keys())
today = date.today().isoformat()

# ---------- Helpers ----------
def read_csv_robust(path):
    """Try utf-8-sig, then utf-16; if still 1 col, manual 'quoted line' split."""
    last_err = None
    for enc in ("utf-8-sig", "utf-16"):
        try:
            tmp = pd.read_csv(path, encoding=enc, sep=",", engine="python")
            if tmp.shape[1] > 1:
                return tmp
        except Exception as e:
            last_err = e
    # Manual fallback for all-in-one quoted column
    with open(path, "r", encoding="utf-8-sig", errors="replace") as f:
        lines = [ln.strip() for ln in f if ln.strip()]
    if not lines:
        raise ValueError(f"Empty file: {path} (last pandas error: {last_err})")
    rows = []
    for ln in lines:
        if ln.startswith('"') and ln.endswith('"'):
            ln = ln[1:-1]
        rows.append(ln.split(","))
    header = [h.strip().strip('"').replace("\ufeff","") for h in rows[0]]
    data = rows[1:]
    return pd.DataFrame(data, columns=header)

def seconds_to_hr_min(seconds):
    try:
        seconds = float(seconds)
        if math.isnan(seconds):
            return ""
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        return f"{hours} hr {minutes} min"
    except Exception:
        return ""

def safe_slug(name: str) -> str:
    base = os.path.basename(name)
    base = os.path.splitext(base)[0]
    base = re.sub(r"[^\w\-]+", "_", base).strip("_")
    return base or "report"

# ---------- Exact renames you requested ----------
RENAME_MAP = {
    "Label": "UserId",
    "sum_actions": "Total Actions",
    "sum_corehome_visittotaltime": "Visit Total Time (seconds)",
}

# Columns we keep (after renaming)
KEEP_COLS = [
    "UserId",
    "Total Actions",
    "Visits",
    "Visit Total Time (seconds)",
    "Avg. Actions In Visit per Visit",
    "Avg. Visit Duration (in seconds) per Visit",
]

# Noise columns to drop if present
DROP_COLS = {"level", "Metadata: segment"}

for idx, SRC in enumerate(names, start=1):
    print(f"\nProcessing {SRC} as week{idx}...")
    try:
        df = read_csv_robust(SRC)

        # (1) Rename exactly as specified (only if present)
        present_renames = {k: v for k, v in RENAME_MAP.items() if k in df.columns}
        df = df.rename(columns=present_renames)

        # (2) Drop noise columns if present
        drop_now = [c for c in df.columns if c in DROP_COLS]
        if drop_now:
            df = df.drop(columns=drop_now, errors="ignore")

        # (3) Keep only the requested columns that actually exist
        keep_now = [c for c in KEEP_COLS if c in df.columns]
        df = df[keep_now].copy()

        # (4) Normalize dtypes
        if "UserId" in df.columns:
            df["UserId"] = pd.to_numeric(df["UserId"], errors="coerce").astype("Int64")
            df = df.dropna(subset=["UserId"]).copy()

        numeric_cols = [
            c for c in ["Total Actions", "Visits",
                        "Visit Total Time (seconds)",
                        "Avg. Actions In Visit per Visit",
                        "Avg. Visit Duration (in seconds) per Visit"]
            if c in df.columns
        ]
        for c in numeric_cols:
            df[c] = pd.to_numeric(df[c], errors="coerce")

        # (5) Pretty duration columns
        if "Visit Total Time (seconds)" in df.columns:
            df["Visit Total Time (hr/min)"] = df["Visit Total Time (seconds)"].apply(seconds_to_hr_min)

        if "Avg. Visit Duration (in seconds) per Visit" in df.columns:
            df["Avg. Visit Duration per Visit (hr/min)"] = df["Avg. Visit Duration (in seconds) per Visit"].apply(seconds_to_hr_min)

        # (6) Order columns nicely
        ordered = [
            "UserId",
            "Total Actions",
            "Visits",
            "Visit Total Time (seconds)",
            "Visit Total Time (hr/min)",
            "Avg. Actions In Visit per Visit",
            "Avg. Visit Duration (in seconds) per Visit",
            "Avg. Visit Duration per Visit (hr/min)",
        ]
        df = df[[c for c in ordered if c in df.columns]]

        # (7) Save & auto-download
        out_xlsx = os.path.join(CLEAN_DIR, f"user_engagement_clean_week{idx}_{today}.xlsx")
        df.to_excel(out_xlsx, index=False)
        files.download(out_xlsx)

        print(f"  ✓ Saved {len(df):,} rows → {out_xlsx}")
        display(df.head(20))

    except Exception as e:
        print(f"  ✗ Skipped {SRC} (week{idx}) due to {type(e).__name__}: {e}")
        continue


Saving outputs to: /content/cleaned


Saving Aug4-Aug10.csv to Aug4-Aug10.csv
Saving Aug11-Aug17.csv to Aug11-Aug17.csv
Saving Aug18-Aug24.csv to Aug18-Aug24.csv
Saving Aug25-Aug31.csv to Aug25-Aug31.csv
Saving Jul14-Jul20.csv to Jul14-Jul20.csv
Saving Jul21-Jul27.csv to Jul21-Jul27.csv
Saving Jul28-Aug3.csv to Jul28-Aug3.csv
Saving Sep1-Sep7.csv to Sep1-Sep7.csv

Processing Aug11-Aug17.csv as week1...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  ✓ Saved 9,998 rows → /content/cleaned/user_engagement_clean_week1_2025-10-03.xlsx


...


Processing Sep1-Sep7.csv as week8...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  ✓ Saved 9,998 rows → /content/cleaned/user_engagement_clean_week8_2025-10-03.xlsx


,UserId,Total Actions,Visits,Visit Total Time (seconds),Visit Total Time (hr/min),Avg. Actions In Visit per Visit,Avg. Visit Duration (in seconds) per Visit,Avg. Visit Duration per Visit (hr/min)
0,435770,3791,312,480399,133 hr 26 min,12.15,1539.74,0 hr 25 min
1,0,1001,238,18792,5 hr 13 min,4.21,78.96,0 hr 1 min
2,469537,950,98,101017,28 hr 3 min,9.69,1030.79,0 hr 17 min
3,10323,877,89,97146,26 hr 59 min,9.85,1091.53,0 hr 18 min
4,34257,797,89,43717,12 hr 8 min,8.96,491.20,0 hr 8 min
5,226091,843,88,65344,18 hr 9 min,9.58,742.55,0 hr 12 min
6,7223,1193,86,55225,15 hr 20 min,13.87,642.15,0 hr 10 min
7,473566,1307,86,95013,26 hr 23 min,15.20,1104.80,0 hr 18 min
8,517656,554,86,67691,18 hr 48 min,6.44,787.10,0 hr 13 min
9,17834,807,85,80683,22 hr 24 min,9.49,949.21,0 hr 15 min


In [ ]:
import os, glob, re
import pandas as pd

CLEAN_DIR = "/content/cleaned"

# Find new engagement files first; fall back to the older category-split pattern
pat_xlsx_new = os.path.join(CLEAN_DIR, "user_engagement_clean_week*_*.xlsx")
pat_csv_new  = os.path.join(CLEAN_DIR, "user_engagement_clean_week*_*.csv")

pat_xlsx_old = os.path.join(CLEAN_DIR, "user_category_split_report_week*_*.xlsx")
pat_csv_old  = os.path.join(CLEAN_DIR, "user_category_split_report_week*_*.csv")

def sort_by_week(files):
    def wknum(f):
        m = re.search(r"week(\d+)", os.path.basename(f))
        return int(m.group(1)) if m else 0
    return sorted(files, key=wknum)

xlsx_files = sort_by_week(glob.glob(pat_xlsx_new) or glob.glob(pat_xlsx_old))
csv_files  = sort_by_week(glob.glob(pat_csv_new)  or glob.glob(pat_csv_old))

files = xlsx_files if xlsx_files else csv_files
if not files:
    raise FileNotFoundError(
        "No cleaned exports found in /content/cleaned. "
        "Expected files like 'user_engagement_clean_weekN_YYYY-MM-DD.xlsx'."
    )

dfs = []
for f in files:
    # Extract the week number from the filename
    m = re.search(r"week(\d+)", os.path.basename(f))
    week_num = int(m.group(1)) if m else None

    # Read Excel or CSV
    df_week = pd.read_excel(f) if f.lower().endswith(".xlsx") else pd.read_csv(f)

    # Ensure numeric UserId (nullable Int64). Engagement reports have no CategoryId.
    if "UserId" in df_week.columns:
        df_week["UserId"] = pd.to_numeric(df_week["UserId"], errors="coerce").astype("Int64")

    # Tag week from filename
    if week_num is not None:
        df_week["Week"] = week_num

    dfs.append(df_week)

# Combine all weeks
all_weeks_df = pd.concat(dfs, ignore_index=True)

# Count distinct weeks per user
user_weeks_active = (
    all_weeks_df[["UserId", "Week"]]
    .dropna(subset=["UserId", "Week"])
    .drop_duplicates()
    .groupby("UserId", as_index=False)["Week"]
    .nunique()
    .rename(columns={"Week": "WeeksActive"})
)

# Make WeeksActive nullable Int64 (no floats, keeps NA if any)
user_weeks_active["WeeksActive"] = user_weeks_active["WeeksActive"].astype("Int64")

# Merge back
all_weeks_df = all_weeks_df.merge(user_weeks_active, on="UserId", how="left")

# Quick preview with Week + WeeksActive highlighted
styled = (
    all_weeks_df.head(20)
    .style
    .apply(
        lambda row: [
            'background-color: lightyellow' if col in ["Week", "WeeksActive"] else ''
            for col in row.index
        ],
        axis=1
    )
)
styled


,UserId,Total Actions,Visits,Visit Total Time (seconds),Visit Total Time (hr/min),Avg. Actions In Visit per Visit,Avg. Visit Duration (in seconds) per Visit,Avg. Visit Duration per Visit (hr/min),Week,WeeksActive
0,435770,3370,302,277926,77 hr 12 min,11.160000,920.280000,0 hr 15 min,1,8
1,502191,734,108,25738,7 hr 8 min,6.800000,238.310000,0 hr 3 min,1,8
2,465820,555,107,36417,10 hr 6 min,5.190000,340.350000,0 hr 5 min,1,8
3,13479,655,99,132368,36 hr 46 min,6.620000,1337.050000,0 hr 22 min,1,8
4,9405,1556,96,95126,26 hr 25 min,16.210000,990.900000,0 hr 16 min,1,8
5,6874,760,91,107249,29 hr 47 min,8.350000,1178.560000,0 hr 19 min,1,8
6,469537,1034,91,128715,35 hr 45 min,11.360000,1414.450000,0 hr 23 min,1,8
7,537438,696,90,50894,14 hr 8 min,7.730000,565.490000,0 hr 9 min,1,8
8,462266,281,89,52222,14 hr 30 min,3.160000,586.760000,0 hr 9 min,1,8
9,17834,798,87,63847,17 hr 44 min,9.170000,733.870000,0 hr 12 min,1,8


In [ ]:
Step 2: Make a Cadency Data Frame

In [ ]:
# === Clean User Cadence Reports (handles UTF-16 TSV + control chars) ===
import os, re, io
import pandas as pd
from google.colab import files
from datetime import date
from IPython.display import display

# ---------- Output dir ----------
BASE_DIR = "/content" if os.path.isdir("/content") else os.getcwd()
CLEAN_DIR = os.path.join(BASE_DIR, "cleaned")
os.makedirs(CLEAN_DIR, exist_ok=True)
print("Saving outputs to:", CLEAN_DIR)

uploaded = files.upload()           # multi-select OK
names = sorted(uploaded.keys())
today = date.today().isoformat()

# ---------- Helpers ----------
CONTROL_CHARS_RE = re.compile(r"[\x00-\x08\x0B-\x0C\x0E-\x1F]")

def strip_control_chars_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [CONTROL_CHARS_RE.sub("", str(c)) for c in df.columns]
    for c in df.columns:
        if pd.api.types.is_string_dtype(df[c]) or df[c].dtype == "object":
            df[c] = df[c].astype(str).map(lambda s: CONTROL_CHARS_RE.sub("", s))
    return df

def read_csv_robust(path):
    # 1) UTF-16 TSV
    try:
        df = pd.read_csv(path, encoding="utf-16", sep="\t", engine="python")
        if df.shape[1] > 1:
            return df
    except Exception:
        pass
    # 2) UTF-16 CSV
    try:
        df = pd.read_csv(path, encoding="utf-16", sep=",", engine="python")
        if df.shape[1] > 1:
            return df
    except Exception:
        pass
    # 3) UTF-8-SIG TSV
    try:
        df = pd.read_csv(path, encoding="utf-8-sig", sep="\t", engine="python")
        if df.shape[1] > 1:
            return df
    except Exception:
        pass
    # 4) UTF-8-SIG CSV
    return pd.read_csv(path, encoding="utf-8-sig", sep=",", engine="python")

def norm(s: str) -> str:
    s = str(s).replace("\xa0", " ")
    s = " ".join(s.strip().split())
    return s

# Canonical names (exact casing we want)
KEEP_ORDER = [
    "UserId",
    "Visits",                                  # <-- added
    "Total Days since last visit",
    "Max Days since last visit",
    "Avg. Days since last visit per Visit",
]

DROP_NORMALIZED = {
    "level",
    "metadata: segment",
    "metadata:: segment",
    "metadata::segment",
    "metadata:segment",
}

for idx, src in enumerate(names, start=1):
    print(f"\nProcessing {src} as week{idx}...")
    try:
        df = read_csv_robust(src)
        df = strip_control_chars_df(df)

        # --- normalize headers (trim/collapse spaces) ---
        col_map = {c: norm(c) for c in df.columns}

        # map to canonical
        rename = {}
        for c, n in col_map.items():
            low = n.lower()
            if low == "label":
                rename[c] = "UserId"
            elif low == "visits":
                rename[c] = "Visits"
            elif low == "total days since last visit":
                rename[c] = "Total Days since last visit"
            elif low == "max days since last visit":
                rename[c] = "Max Days since last visit"
            elif low == "avg. days since last visit per visit":
                rename[c] = "Avg. Days since last visit per Visit"

        if rename:
            df = df.rename(columns=rename)

        # drop noise columns by normalized name
        to_drop = [c for c in df.columns if norm(c).lower() in DROP_NORMALIZED]
        if to_drop:
            df = df.drop(columns=to_drop, errors="ignore")

        # keep only required columns that exist
        keep_now = [c for c in KEEP_ORDER if c in df.columns]
        if not keep_now:
            print("⚠️ Expected cadence columns not found after normalization; keeping all for inspection.")
            print("Columns:", list(df.columns))
            keep_now = list(df.columns)
        df = df[keep_now].copy()

        # dtypes: UserId as nullable Int64 if mostly numeric, else keep as string
        if "UserId" in df.columns:
            tmp = pd.to_numeric(df["UserId"], errors="coerce")
            nonnull = df["UserId"].notna().sum()
            numeric = tmp.notna().sum()
            if nonnull > 0 and numeric / max(nonnull, 1) >= 0.8:
                df["UserId"] = tmp.astype("Int64")
                df = df.dropna(subset=["UserId"]).copy()
            else:
                df["UserId"] = df["UserId"].astype(str).str.strip()

        # metrics to numeric when present (✅ fixed missing comma)
        for c in [
            "Visits",
            "Total Days since last visit",
            "Max Days since last visit",
            "Avg. Days since last visit per Visit",
        ]:
            if c in df.columns:
                df[c] = pd.to_numeric(df[c], errors="coerce")

        # final column order
        ordered = [c for c in KEEP_ORDER if c in df.columns]
        if ordered:
            df = df[ordered]

        out_xlsx = os.path.join(CLEAN_DIR, f"user_cadence_clean_week{idx}_{today}.xlsx")
        df.to_excel(out_xlsx, index=False)
        files.download(out_xlsx)

        print(f"  ✓ Saved {len(df):,} rows → {out_xlsx}")
        display(df.head(20))

    except Exception as e:
        print(f"  ✗ Skipped {src} (week{idx}) due to {type(e).__name__}: {e}")


Saving outputs to: /content/cleaned


Saving Export _ User Cadence Report _ week August 4 – 10, 2025.csv to Export _ User Cadence Report _ week August 4 – 10, 2025.csv
Saving Export _ User Cadence Report _ week August 11 – 17, 2025.csv to Export _ User Cadence Report _ week August 11 – 17, 2025.csv
Saving Export _ User Cadence Report _ week August 18 – 24, 2025.csv to Export _ User Cadence Report _ week August 18 – 24, 2025.csv
Saving Export _ User Cadence Report _ week August 25 – 31, 2025.csv to Export _ User Cadence Report _ week August 25 – 31, 2025.csv
Saving Export _ User Cadence Report _ week July 14 – 20, 2025.csv to Export _ User Cadence Report _ week July 14 – 20, 2025.csv
Saving Export _ User Cadence Report _ week July 21 – 27, 2025.csv to Export _ User Cadence Report _ week July 21 – 27, 2025.csv
Saving Export _ User Cadence Report _ week July 28 – August 3, 2025.csv to Export _ User Cadence Report _ week July 28 – August 3, 2025.csv
Saving Export _ User Cadence Report _ week September 1 – 7, 2025.csv to Export

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  ✓ Saved 9,998 rows → /content/cleaned/user_cadence_clean_week1_2025-10-03.xlsx


...


Processing Export _ User Cadence Report _ week September 1 – 7, 2025.csv as week8...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  ✓ Saved 9,998 rows → /content/cleaned/user_cadence_clean_week8_2025-10-03.xlsx


,UserId,Visits,Total Days since last visit,Max Days since last visit,Avg. Days since last visit per Visit
0,435770,312,0,0,0.00
1,0,238,80,43,0.34
2,469537,98,0,0,0.00
3,10323,89,0,0,0.00
4,34257,89,0,0,0.00
5,226091,88,0,0,0.00
6,7223,86,0,0,0.00
7,473566,86,0,0,0.00
8,517656,86,0,0,0.00
9,17834,85,0,0,0.00


In [ ]:
# === Combine cleaned CADENCE files → all_weeks_df_cadence ===
import os, glob, re
import pandas as pd

CLEAN_DIR = "/content/cleaned"

# Look for cleaned CADENCE files only
pat_xlsx_cad = os.path.join(CLEAN_DIR, "user_cadence_clean_week*_*.xlsx")
pat_csv_cad  = os.path.join(CLEAN_DIR, "user_cadence_clean_week*_*.csv")

def sort_by_week(files):
    def wknum(f):
        m = re.search(r"week(\d+)", os.path.basename(f))
        return int(m.group(1)) if m else 0
    return sorted(files, key=wknum)

xlsx_files_cad = sort_by_week(glob.glob(pat_xlsx_cad))
csv_files_cad  = sort_by_week(glob.glob(pat_csv_cad))

files_cad = xlsx_files_cad if xlsx_files_cad else csv_files_cad
if not files_cad:
    raise FileNotFoundError(
        "No cleaned cadence exports found in /content/cleaned. "
        "Expected files like 'user_cadence_clean_weekN_YYYY-MM-DD.xlsx'."
    )

dfs_cadence = []
for f in files_cad:
    # Extract the week number from the filename
    m = re.search(r"week(\d+)", os.path.basename(f))
    week_num = int(m.group(1)) if m else None

    # Read Excel or CSV
    df_week_cad = pd.read_excel(f) if f.lower().endswith(".xlsx") else pd.read_csv(f)

    # Ensure numeric UserId (nullable Int64)
    if "UserId" in df_week_cad.columns:
        df_week_cad["UserId"] = pd.to_numeric(df_week_cad["UserId"], errors="coerce").astype("Int64")

    # Tag Week from filename
    if week_num is not None:
        df_week_cad["Week"] = week_num

    dfs_cadence.append(df_week_cad)

# Combine all cadence weeks
all_weeks_df_cadence = pd.concat(dfs_cadence, ignore_index=True)

# Count distinct weeks per user (cadence)
user_weeks_active_cadence = (
    all_weeks_df_cadence[["UserId", "Week"]]
    .dropna(subset=["UserId", "Week"])
    .drop_duplicates()
    .groupby("UserId", as_index=False)["Week"]
    .nunique()
    .rename(columns={"Week": "WeeksActive"})
)

# Keep nullable Int64
user_weeks_active_cadence["WeeksActive"] = user_weeks_active_cadence["WeeksActive"].astype("Int64")

# Merge back
all_weeks_df_cadence = all_weeks_df_cadence.merge(user_weeks_active_cadence, on="UserId", how="left")

# Quick preview with Week + WeeksActive highlighted
styled_cadence = (
    all_weeks_df_cadence.head(20)
    .style
    .apply(
        lambda row: [
            'background-color: lightyellow' if col in ["Week", "WeeksActive"] else ''
            for col in row.index
        ],
        axis=1
    )
)
styled_cadence


,UserId,Visits,Total Days since last visit,Max Days since last visit,Avg. Days since last visit per Visit,Week,WeeksActive
0,435770,302,0,0,0.000000,1,8
1,502191,108,0,0,0.000000,1,8
2,465820,107,0,0,0.000000,1,8
3,13479,99,2,1,0.020000,1,8
4,9405,96,0,0,0.000000,1,8
5,6874,91,0,0,0.000000,1,8
6,469537,91,0,0,0.000000,1,8
7,537438,90,0,0,0.000000,1,8
8,462266,89,0,0,0.000000,1,8
9,17834,87,0,0,0.000000,1,8


Combine & Clean the Merged Data Frame

In [ ]:
import pandas as pd

eng = all_weeks_df.copy()
cad = all_weeks_df_cadence.copy()

# Choose keys
join_keys = ["UserId", "Week"] if ("Week" in eng.columns and "Week" in cad.columns) else ["UserId"]

# Full OUTER join
merged = eng.merge(
    cad,
    on=join_keys,
    how="outer",
    suffixes=("_eng", "_cad")
)

# Coalesce duplicate columns (those present in both, excluding join keys)
dup_bases = []
for c in merged.columns:
    if c.endswith("_eng"):
        base = c[:-4]
        if f"{base}_cad" in merged.columns and base not in join_keys:
            dup_bases.append(base)

for base in dup_bases:
    le, rc = f"{base}_eng", f"{base}_cad"
    if base == "WeeksActive":
        # take the max across sources
        merged[base] = merged[[le, rc]].max(axis=1)
    else:
        # prefer engagement, fallback to cadence
        merged[base] = merged[le].combine_first(merged[rc])
    merged.drop(columns=[le, rc], inplace=True)

# Optional: enforce nullable Int64 where appropriate
for col in ["UserId", "Week", "WeeksActive", "Visits"]:
    if col in merged.columns:
        merged[col] = pd.to_numeric(merged[col], errors="coerce").astype("Int64")

# Nice column order: keys first
key_cols = [k for k in join_keys if k in merged.columns]
other_cols = [c for c in merged.columns if c not in key_cols]
merged = merged[key_cols + other_cols]

merged.head(20)

merged.to_excel("/content/merged_engagement_cadence.xlsx", index=False)


In [ ]:
# === Add "Subscribed" flag to merged by uploading Current Subscribers, highlight & export ===

# (Optional) Ensure Excel writer is available in Colab/Jupyter:
# If you're in Colab, this line will install xlsxwriter; harmless elsewhere.
!pip install -q xlsxwriter

import io, re, os, sys
import pandas as pd

# Detect Colab for upload flow
try:
    from google.colab import files  # type: ignore
    _IN_COLAB = True
except Exception:
    _IN_COLAB = False

# ---------- Helpers ----------
def soft_read_any(path_or_bytes):
    """Read Excel/CSV/TSV from path or bytes, trying Excel -> CSV -> TSV."""
    if isinstance(path_or_bytes, (bytes, bytearray)):
        buf = io.BytesIO(path_or_bytes)
        for reader in (pd.read_excel, pd.read_csv):
            try:
                buf.seek(0)
                return reader(buf)
            except Exception:
                pass
        buf.seek(0)
        return pd.read_csv(buf, sep="\t")

    p = str(path_or_bytes)
    pl = p.lower()
    if pl.endswith((".xlsx", ".xls")):
        return pd.read_excel(p)
    if pl.endswith(".tsv"):
        return pd.read_csv(p, sep="\t")
    return pd.read_csv(p)

def find_user_col(cols):
    """Find a UserId-like column by exacts first, then fuzzy."""
    exacts = ["UserId","user_id","userid","uid","id","User ID","userId"]
    for c in exacts:
        if c in cols:
            return c
    for c in cols:
        if re.search(r'user\s*id', str(c), flags=re.I):
            return c
    return None

def excel_col_letter(idx_zero_based: int) -> str:
    """Convert 0-based column index to Excel column letter(s)."""
    n = idx_zero_based + 1
    letters = []
    while n > 0:
        n, rem = divmod(n - 1, 26)
        letters.append(chr(ord('A') + rem))
    return "".join(reversed(letters))

# ---------- 1) Load subscribers file (Excel/CSV/TSV) ----------
if _IN_COLAB:
    print("Upload your CURRENT SUBSCRIBERS file (Excel/CSV/TSV).")
    up = files.upload()  # you can multi-select; we'll use the first
    if not up:
        raise RuntimeError("No file uploaded.")
    sub_name, sub_bytes = next(iter(up.items()))
    subs_df = soft_read_any(sub_bytes)
    print(f"Loaded subscribers: {sub_name} ({len(subs_df):,} rows)")
else:
    # Not in Colab? Set a local path here:
    SUBSCRIBERS_PATH = "./current_subscribers.xlsx"  # change if needed
    if not os.path.exists(SUBSCRIBERS_PATH):
        raise FileNotFoundError(
            "Set SUBSCRIBERS_PATH to your subscribers file path (Excel/CSV/TSV)."
        )
    subs_df = soft_read_any(SUBSCRIBERS_PATH)
    print(f"Loaded subscribers: {SUBSCRIBERS_PATH} ({len(subs_df):,} rows)")

# ---------- 2) Normalize UserId and build a lookup set ----------
uid_col = find_user_col(list(subs_df.columns))
if uid_col is None:
    raise KeyError("Could not detect a UserId-like column in the subscribers file.")
if uid_col != "UserId":
    subs_df = subs_df.rename(columns={uid_col: "UserId"})

# Drop NaNs BEFORE astype(str) to avoid 'nan' strings
subs_ids = set(subs_df["UserId"].dropna().astype(str))

# ---------- 3) Add Subscribed flag to 'merged' ----------
if "merged" not in globals():
    raise RuntimeError("`merged` dataframe not found. Run the merge cell first.")

# Ensure merged has a UserId column (rename if necessary)
m_uid_col = find_user_col(list(merged.columns))
if m_uid_col is None:
    raise KeyError("Could not detect a UserId-like column in the merged dataframe.")
if m_uid_col != "UserId":
    merged = merged.rename(columns={m_uid_col: "UserId"})

merged["Subscribed"] = merged["UserId"].astype(str).isin(subs_ids).map(
    {True: "Subscribed", False: "Not Subscribed"}
)

# ---------- 4) Optional: keep any status/plan columns if present ----------
keep_cols = [c for c in ["status","Status","SubscriberStatus","plan","Plan","SubscriberPlan"] if c in subs_df.columns]
if keep_cols:
    subs_small = subs_df[["UserId"] + keep_cols].drop_duplicates(subset="UserId")
    merged = merged.merge(subs_small, on="UserId", how="left")

# ---------- 5) Display with highlighting (subscribers in yellow) ----------
def _highlight_row(s):
    color = '#fff3b0' if (isinstance(s.get("Subscribed", None), str) and s["Subscribed"] == "Subscribed") else ''
    return [f'background-color: {color}' if color else '' for _ in s.index]

try:
    from IPython.display import display  # type: ignore
    display(merged.head(20).style.apply(_highlight_row, axis=1))
except Exception:
    # Fallback if not in a rich display env
    print(merged.head(20).to_string(index=False))

# ---------- 6) Export to Excel with conditional formatting ----------
out_path = ("/content/merged_engagement_cadence_with_subscribers.xlsx"
            if _IN_COLAB else
            "./merged_engagement_cadence_with_subscribers.xlsx")

with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
    merged.to_excel(writer, index=False, sheet_name="merged")
    ws = writer.sheets["merged"]
    workbook = writer.book
    fmt_sub = workbook.add_format({"bg_color": "#FFF3B0"})

    # Compute the full range (A2 : <last_col><last_row>)
    n_cols = len(merged.columns)
    n_rows = len(merged) + 1  # +1 because header is row 1 in Excel
    first_data_row = 2        # data starts at row 2
    last_row = n_rows
    first_col_letter = excel_col_letter(0)
    last_col_letter = excel_col_letter(n_cols - 1)

    # Find the column letter for "Subscribed"
    sub_col_idx = merged.columns.get_loc("Subscribed")
    sub_col_letter = excel_col_letter(sub_col_idx)

    # Apply conditional formatting across the whole data range.
    # Use a relative row formula that locks the column: =$<sub_col>2="Subscribed"
    ws.conditional_format(
        f"{first_col_letter}{first_data_row}:{last_col_letter}{last_row}",
        {
            "type": "formula",
            "criteria": f'=${sub_col_letter}{first_data_row}="Subscribed"',
            "format": fmt_sub,
        },
    )

print(f"Saved with highlights → {out_path}")


Upload your CURRENT SUBSCRIBERS file (Excel/CSV/TSV).


Saving Currently Subscribed List.xlsx to Currently Subscribed List.xlsx
Loaded subscribers: Currently Subscribed List.xlsx (10,894 rows)


,UserId,Week,Total Actions,Visit Total Time (seconds),Visit Total Time (hr/min),Avg. Actions In Visit per Visit,Avg. Visit Duration (in seconds) per Visit,Avg. Visit Duration per Visit (hr/min),Total Days since last visit,Max Days since last visit,Avg. Days since last visit per Visit,Visits,WeeksActive,Subscribed
0,0,1,483,17061,4 hr 44 min,10.500000,370.890000,0 hr 6 min,191,180,4.150000,46,8,Not Subscribed
1,0,2,1225,44634,12 hr 23 min,5.470000,199.260000,0 hr 3 min,271,64,1.210000,224,8,Not Subscribed
2,0,3,1782,30844,8 hr 34 min,2.730000,47.310000,0 hr 0 min,346,319,0.530000,652,8,Not Subscribed
3,0,4,575,16125,4 hr 28 min,15.130000,424.340000,0 hr 7 min,184,71,4.840000,38,8,Not Subscribed
4,0,5,716,17375,4 hr 49 min,21.060000,511.030000,0 hr 8 min,70,25,2.060000,34,8,Not Subscribed
5,0,6,533,15224,4 hr 13 min,11.840000,338.310000,0 hr 5 min,230,56,5.110000,45,8,Not Subscribed
6,0,7,746,23084,6 hr 24 min,12.860000,398.000000,0 hr 6 min,622,380,10.720000,58,8,Not Subscribed
7,0,8,1001,18792,5 hr 13 min,4.210000,78.960000,0 hr 1 min,80,43,0.340000,238,8,Not Subscribed
8,1,1,84,9169,2 hr 32 min,12.000000,1309.860000,0 hr 21 min,4,4,0.570000,7,8,Subscribed
9,1,2,92,2269,0 hr 37 min,9.200000,226.900000,0 hr 3 min,4,2,0.400000,10,8,Subscribed


Saved with highlights → /content/merged_engagement_cadence_with_subscribers.xlsx


In [ ]:
# === Add "New Accounts" flag to `merged` using a New Accounts file; Creation date aware; highlight & export ===
# Drop-in Colab cell (fixed regex)

# If you're not in Colab, set NEW_ACCOUNTS_PATH below.
# Assumes you already created a `merged` DataFrame earlier in your notebook.

import io, os, re, unicodedata
import numpy as np
import pandas as pd
from IPython.display import display

# (Colab-friendly; harmless elsewhere)
try:
    import google.colab  # noqa
    _IN_COLAB = True
except Exception:
    _IN_COLAB = False

# ---- optional writer ----
try:
    import xlsxwriter  # noqa
except Exception:
    import sys, subprocess
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "xlsxwriter"], check=False)

# ---------- helpers ----------
def soft_read_any(path_or_bytes):
    """Read Excel/CSV/TSV from a path or uploaded bytes."""
    if isinstance(path_or_bytes, (bytes, bytearray)):
        buf = io.BytesIO(path_or_bytes)
        for reader in (
            pd.read_excel,
            lambda b: pd.read_csv(io.BytesIO(path_or_bytes)),
            lambda b: pd.read_csv(io.BytesIO(path_or_bytes), sep="\t"),
        ):
            try:
                buf.seek(0)
                return reader(buf)
            except Exception:
                continue
        raise ValueError("Could not read uploaded file.")
    p = str(path_or_bytes)
    pl = p.lower()
    if pl.endswith((".xlsx", ".xls")):
        return pd.read_excel(p)
    if pl.endswith(".tsv"):
        return pd.read_csv(p, sep="\t")
    return pd.read_csv(p)

def _norm(s: str) -> str:
    """lowercase, strip, collapse spaces, remove accents & punctuation."""
    if s is None:
        return ""
    s = str(s)
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"[^\w\s]", " ", s, flags=re.U)
    s = re.sub(r"\s+", " ", s).strip().lower()
    return s

def find_user_col(cols):
    """Find a UserId-like column (handles EN/FR variants)."""
    wanted = [
        "userid","user id","user_id","id","uid","userId".lower(),
        "identifiant utilisateur","id utilisateur"
    ]
    norm_map = {_norm(c): c for c in cols}
    for target in wanted:
        if target in norm_map:
            return norm_map[target]
    # fuzzy
    for c in cols:
        if re.search(r"\buser\s*id\b", _norm(c)):
            return c
    return None

def find_datecreated_col(cols):
    """
    Prefer a 'Creation date' style column from the report.
    Handles: 'Creation date', 'CreationDate', 'creation_date',
    'Date created', 'Created at', and FR variants like 'Date de création'.
    Avoids grabbing 'Subscription expiry' etc.
    """
    norm_map = {_norm(c): c for c in cols}

    strong = [
        "creation date","creationdate","creation_date",
        "date created","datecreated","date_created",
        "created at","created_at","createdat",
        # french
        "date de creation","cree le","date de creation du compte",
    ]
    for key in strong:
        if key in norm_map:
            return norm_map[key]

    # careful fuzzy: needs a creation word AND a date word
    for c in cols:
        n = _norm(c)
        has_creation = bool(re.search(r"\b(creation|created|cree)\b", n))
        has_date     = bool(re.search(r"\b(date|jour|datetime)\b", n))
        if has_creation and has_date:
            return c

    # last resort: contains a creation keyword but not obviously unrelated fields
    for c in cols:
        n = _norm(c)
        if any(k in n for k in ["creation","created","cree"]):
            if not any(bad in n for bad in ["subscription", "expiry", "expire", "group"]):
                return c

    return None

def excel_colname(col_idx_zero_based: int) -> str:
    n = col_idx_zero_based + 1
    name = ""
    while n > 0:
        n, rem = divmod(n - 1, 26)
        name = chr(65 + rem) + name
    return name

# ---------- 1) Load "New Accounts" file (Excel/CSV/TSV) ----------
if _IN_COLAB:
    from google.colab import files
    print("Upload your NEW ACCOUNTS file (Excel/CSV/TSV).")
    up = files.upload()
    if not up:
        raise RuntimeError("No file uploaded.")
    new_name, new_bytes = next(iter(up.items()))
    new_df = soft_read_any(new_bytes)
    print(f"Loaded new-accounts file: {new_name} ({len(new_df):,} rows)")
else:
    NEW_ACCOUNTS_PATH = "/content/new_accounts.xlsx"  # <-- change if needed
    if not os.path.exists(NEW_ACCOUNTS_PATH):
        raise FileNotFoundError("Set NEW_ACCOUNTS_PATH to your new-accounts file path.")
    new_df = soft_read_any(NEW_ACCOUNTS_PATH)
    print(f"Loaded new-accounts file: {NEW_ACCOUNTS_PATH} ({len(new_df):,} rows)")

# ---------- 2) Normalize UserId from the New Accounts file -> set ----------
uid_col = find_user_col(list(new_df.columns))
if uid_col is None:
    raise KeyError("Could not detect a UserId-like column in the new-accounts file.")
new_df = new_df.rename(columns={uid_col: "UserId"})
new_ids = set(new_df["UserId"].astype(str).dropna())

# ---------- 3) Preconditions on `merged` ----------
if "merged" not in globals():
    raise RuntimeError("`merged` dataframe not found. Run the cell that creates `merged` first.")
if "UserId" not in merged.columns:
    ucol = find_user_col(list(merged.columns))
    if ucol:
        merged = merged.rename(columns={ucol: "UserId"})
    else:
        raise KeyError("`merged` must have a UserId-like column.")

# ---------- 4) Compute New/Old with the 20-day rule ----------
cond_id = merged["UserId"].astype(str).isin(new_ids)

date_col = find_datecreated_col(list(merged.columns))
if date_col:
    # Prefer day-first parsing (common in EU/ME exports); fallback to default.
    dc = pd.to_datetime(merged[date_col], errors="coerce", dayfirst=True, utc=True)
    if dc.isna().all():
        dc = pd.to_datetime(merged[date_col], errors="coerce", utc=True)
    now = pd.Timestamp.utcnow()
    age_days = (now - dc).dt.days
    cond_recent = age_days < 20
else:
    cond_recent = True  # fallback if no date

merged["New Accounts"] = np.where(cond_id & cond_recent, "New", "Old")

# helpful logs
if not date_col:
    print("Note: No 'Creation date' style column found in `merged`; fell back to ID-only classification.")
else:
    print(f"Using creation-date column: {date_col}")
    missing_dates = merged[date_col].isna().sum()
    if missing_dates:
        print(f"Info: {missing_dates} rows have missing/unparseable {date_col} and were marked 'Old'.")

# ---------- 5) Display with highlighting (NEW accounts in yellow) ----------
def _hl(row):
    return ['background-color: #FFF3B0' if row.get("New Accounts") == "New" else '' for _ in row]

try:
    display(merged.head(20).style.apply(_hl, axis=1))
except Exception:
    display(merged.head(20))

# ---------- 6) Export to Excel with conditional formatting ----------
out_path = "/content/merged_engagement_cadence_with_new_accounts.xlsx"
with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
    merged.to_excel(writer, index=False, sheet_name="merged")
    ws = writer.sheets["merged"]
    workbook  = writer.book
    fmt_new   = workbook.add_format({"bg_color": "#FFF3B0"})

    new_col_idx = merged.columns.get_loc("New Accounts")
    new_col_letter = excel_colname(new_col_idx)
    last_row = len(merged) + 1  # header offset
    last_col_letter = excel_colname(len(merged.columns) - 1)

    # Highlight rows where New Accounts == "New"
    ws.conditional_format(
        f"A2:{last_col_letter}{last_row}",
        {"type": "formula", "criteria": f'=${new_col_letter}2="New"', "format": fmt_new}
    )

print(f"Saved with highlights → {out_path}")


Upload your NEW ACCOUNTS file (Excel/CSV/TSV).


Saving New Accounts.csv to New Accounts.csv
Loaded new-accounts file: New Accounts.csv (975 rows)
Note: No 'Creation date' style column found in `merged`; fell back to ID-only classification.


,UserId,Week,Total Actions,Visit Total Time (seconds),Visit Total Time (hr/min),Avg. Actions In Visit per Visit,Avg. Visit Duration (in seconds) per Visit,Avg. Visit Duration per Visit (hr/min),Total Days since last visit,Max Days since last visit,Avg. Days since last visit per Visit,Visits,WeeksActive,Subscribed,New Accounts
0,0,1,483,17061,4 hr 44 min,10.500000,370.890000,0 hr 6 min,191,180,4.150000,46,8,Not Subscribed,Old
1,0,2,1225,44634,12 hr 23 min,5.470000,199.260000,0 hr 3 min,271,64,1.210000,224,8,Not Subscribed,Old
2,0,3,1782,30844,8 hr 34 min,2.730000,47.310000,0 hr 0 min,346,319,0.530000,652,8,Not Subscribed,Old
3,0,4,575,16125,4 hr 28 min,15.130000,424.340000,0 hr 7 min,184,71,4.840000,38,8,Not Subscribed,Old
4,0,5,716,17375,4 hr 49 min,21.060000,511.030000,0 hr 8 min,70,25,2.060000,34,8,Not Subscribed,Old
5,0,6,533,15224,4 hr 13 min,11.840000,338.310000,0 hr 5 min,230,56,5.110000,45,8,Not Subscribed,Old
6,0,7,746,23084,6 hr 24 min,12.860000,398.000000,0 hr 6 min,622,380,10.720000,58,8,Not Subscribed,Old
7,0,8,1001,18792,5 hr 13 min,4.210000,78.960000,0 hr 1 min,80,43,0.340000,238,8,Not Subscribed,Old
8,1,1,84,9169,2 hr 32 min,12.000000,1309.860000,0 hr 21 min,4,4,0.570000,7,8,Subscribed,Old
9,1,2,92,2269,0 hr 37 min,9.200000,226.900000,0 hr 3 min,4,2,0.400000,10,8,Subscribed,Old


Saved with highlights → /content/merged_engagement_cadence_with_new_accounts.xlsx


Analyze the combined Data Frames

In [ ]:
# === Engagement score with MAD-based repair on Avg Time/Visit (uses `merged`) ===
import numpy as np
import pandas as pd
from datetime import date
from google.colab import files

# ----------------- CONFIG (tweak as needed) -----------------
# Weights (must sum to 1).
W_ACTIONS = 0.6     # actions per visit
W_TIME    = 0.35    # average duration per visit (after repair)
W_VISITS  = 0.05    # tiny weight for visits; cadence will use visits anyway

# Robust scaling caps (percentiles for min/max clipping before 0..1 scaling)
ROBUST_LO, ROBUST_HI = 5, 95

# MAD outlier rule for Avg Time/Visit
MAD_K            = 3.5   # how many MADs above median counts as inflated
REPLACE_WITH_PCT = 20    # replace outliers with this percentile of the distribution (P20)

# ----------------- Column discovery (flexible headers) -----------------
def pick(cols, candidates):
    for c in candidates:
        if c in cols:
            return c
    return None

cols = merged.columns

# engagement columns (post-merge, usually from the engagement side)
COL_USER   = pick(cols, ["UserId"])
COL_WEEK   = pick(cols, ["Week"])

COL_VISITS = pick(cols, [
    "Visits", "Visits_eng", "Visits_cad"  # coalesced name or suffixed, depending on your join
])

# Avg Actions per Visit
COL_APV = pick(cols, [
    "Avg. Actions In Visit per Visit",
    "Avg Actions in Visit Per Visit",
    "Avg. Actions per Visit",
    "Avg.ActionsInVisitperVisit",
    "Avg Actions per Visit_eng", "Avg Actions per Visit_cad"
])

# Avg Visit Duration (seconds) per Visit
COL_AVD = pick(cols, [
    "Avg. Visit Duration (in seconds) per Visit",
    "Avg Visit Duration (seconds)",
    "Avg Visit Duration (in seconds) per Visit",
    "Avg.VisitDuration(inseconds)perVisit",
    "Avg Visit Duration per Visit (seconds)"
])

# Fallbacks if APV or AVD not present but we have totals
COL_ACTIONS_TOTAL = pick(cols, ["Total Actions", "sum_actions", "sum_actions_eng"])
COL_TT_SECONDS    = pick(cols, ["Visit Total Time (seconds)", "Visit Total Time (seconds)_eng"])

# ----------------- Build working copy and coerce numerics -----------------
eng_df = merged.copy()

# numeric coercions (silent)
for c in [COL_VISITS, COL_APV, COL_AVD, COL_ACTIONS_TOTAL, COL_TT_SECONDS]:
    if c and c in eng_df.columns:
        eng_df[c] = pd.to_numeric(eng_df[c], errors="coerce")

# Derive APV/AVD if missing and we have totals
if COL_APV is None and COL_ACTIONS_TOTAL and COL_VISITS:
    eng_df["__apv_fallback"] = eng_df[COL_ACTIONS_TOTAL] / eng_df[COL_VISITS]
    COL_APV = "__apv_fallback"

if COL_AVD is None and COL_TT_SECONDS and COL_VISITS:
    eng_df["__avd_fallback"] = eng_df[COL_TT_SECONDS] / eng_df[COL_VISITS]
    COL_AVD = "__avd_fallback"

# ----------------- MAD-based repair on Avg Time per Visit -----------------
def mad_repair(series: pd.Series, k=MAD_K, replace_pct=REPLACE_WITH_PCT):
    s = series.astype(float).copy()
    # median & MAD
    med = np.nanmedian(s)
    mad = np.nanmedian(np.abs(s - med))
    # scale MAD to sigma (consistency constant)
    sigma = 1.4826 * mad if mad > 0 else 0.0
    # threshold: above this => suspicious (inflated)
    upper = med + k * sigma if sigma > 0 else np.inf

    # replacement value: chosen lower percentile (P20 by default)
    p_lo = np.nanpercentile(s, replace_pct) if s.notna().any() else med

    repaired = s.copy()
    flagged  = s > upper
    repaired[flagged] = p_lo
    return repaired, flagged, {"median": med, "mad": mad, "sigma": sigma, "upper": upper, "p_lo": p_lo}

# Only repair if we have an AVD column
if COL_AVD:
    eng_df["AvgTimePerVisit_sec_raw"] = eng_df[COL_AVD]
    eng_df["AvgTimePerVisit_sec_repair"], avd_flag, avd_stats = mad_repair(eng_df[COL_AVD])
    eng_df["flag_avd_inflated"] = avd_flag.fillna(False)
else:
    eng_df["AvgTimePerVisit_sec_raw"] = np.nan
    eng_df["AvgTimePerVisit_sec_repair"] = np.nan
    eng_df["flag_avd_inflated"] = False
    avd_stats = {"median": np.nan, "mad": np.nan, "sigma": np.nan, "upper": np.nan, "p_lo": np.nan}

# ----------------- Robust min-max scaling (percentile-based) -----------------
def robust_minmax(s, lo=ROBUST_LO, hi=ROBUST_HI):
    if s is None or s.isna().all():
        return pd.Series(np.zeros(len(s)), index=s.index)
    a = np.nanpercentile(s, lo)
    b = np.nanpercentile(s, hi)
    if not np.isfinite(a) or not np.isfinite(b) or b == a:
        return pd.Series(np.zeros(len(s)), index=s.index)
    return ((s.clip(a, b) - a) / (b - a))

# Build components
apv = eng_df[COL_APV] if COL_APV else pd.Series(np.nan, index=eng_df.index)
avd = eng_df["AvgTimePerVisit_sec_repair"]
vis = eng_df[COL_VISITS] if COL_VISITS else pd.Series(np.nan, index=eng_df.index)

# Scale each component to 0..1 robustly
apv_scaled = robust_minmax(apv)
avd_scaled = robust_minmax(avd)
vis_scaled = robust_minmax(vis)

eng_df["apv_scaled"] = apv_scaled
eng_df["avd_scaled"] = avd_scaled
eng_df["vis_scaled"] = vis_scaled

# ----------------- EngagementScore -----------------
# Ensure weights sum to 1 even if you tweak config
w_sum = W_ACTIONS + W_TIME + W_VISITS
wa, wt, wv = W_ACTIONS / w_sum, W_TIME / w_sum, W_VISITS / w_sum

eng_df["EngagementScore"] = wa * eng_df["apv_scaled"] + wt * eng_df["avd_scaled"] + wv * eng_df["vis_scaled"]

# ----------------- Output tidy view -----------------
out_cols = [c for c in [COL_USER, COL_WEEK] if c] + [
    "EngagementScore",
    COL_APV if COL_APV else None,
    "AvgTimePerVisit_sec_raw","AvgTimePerVisit_sec_repair","flag_avd_inflated",
    COL_VISITS if COL_VISITS else None,
]
out_cols = [c for c in out_cols if c in eng_df.columns and c is not None]

eng_tidy = eng_df[out_cols].copy().sort_values("EngagementScore", ascending=False)

# Save & (optional) download
out_path = f"/content/cleaned/engagement_scored_{date.today().isoformat()}.xlsx"
eng_tidy.to_excel(out_path, index=False)
# from google.colab import files
# files.download(out_path)

print("MAD repair stats for AvgTimePerVisit (seconds):", avd_stats)
eng_tidy.head(20)


MAD repair stats for AvgTimePerVisit (seconds): {'median': np.float64(249.95999999999998), 'mad': np.float64(191.95999999999998), 'sigma': np.float64(284.59989599999994), 'upper': np.float64(1246.0596359999997), 'p_lo': np.float64(64.0)}


,UserId,Week,EngagementScore,Avg. Actions In Visit per Visit,AvgTimePerVisit_sec_raw,AvgTimePerVisit_sec_repair,flag_avd_inflated,Visits
22246,437656,3,1.0,26.40,988.02,988.02,False,45
21946,437377,1,1.0,25.02,1003.85,1003.85,False,47
46584,499765,1,1.0,27.72,1088.35,1088.35,False,43
67923,552961,7,1.0,26.46,972.97,972.97,False,63
46686,499848,5,1.0,24.07,1219.93,1219.93,False,43
46587,499765,4,1.0,24.21,1164.16,1164.16,False,43
49817,507625,1,1.0,25.53,1206.98,1206.98,False,85
21952,437377,7,1.0,23.83,901.67,901.67,False,48
2846,10087,5,1.0,30.61,908.04,908.04,False,56
27421,460604,5,1.0,24.16,1205.02,1205.02,False,45


In [ ]:
# === Engagement score (reuse pre-repaired AVD if present; else MAD-repair) ===
import numpy as np
import pandas as pd
from datetime import date
from google.colab import files
from IPython.display import display

# Config
W_ACTIONS, W_TIME, W_VISITS = 0.60, 0.35, 0.05
ROBUST_LO, ROBUST_HI = 5, 95
MAD_K, REPLACE_WITH_PCT = 3.5, 20

def pick(cols, candidates):
    for c in candidates:
        if c in cols: return c
    return None

eng_df = merged.copy()
cols = eng_df.columns

COL_USER   = pick(cols, ["UserId"])
COL_WEEK   = pick(cols, ["Week"])
COL_VISITS = pick(cols, ["Visits","Visits_eng","Visits_cad"])
COL_APV    = pick(cols, [
    "Avg. Actions In Visit per Visit","Avg Actions in Visit Per Visit",
    "Avg. Actions per Visit","Avg.ActionsInVisitperVisit",
    "Avg. Actions In Visit per Visit_eng","Avg. Actions In Visit per Visit_cad"
])
COL_AVD    = pick(cols, [
    "Avg. Visit Duration (in seconds) per Visit","Avg Visit Duration (seconds)",
    "Avg Visit Duration (in seconds) per Visit","Avg.VisitDuration(inseconds)perVisit",
    "Avg Visit Duration per Visit (seconds)"
])

# Fallback totals (only used if APV/AVD are missing)
COL_ACTIONS_TOTAL = pick(cols, ["Total Actions","sum_actions","sum_actions_eng"])
COL_TT_SECONDS    = pick(cols, ["Visit Total Time (seconds)","Visit Total Time (seconds)_eng"])

# Coerce numerics softly
for c in [COL_VISITS, COL_APV, COL_AVD, COL_ACTIONS_TOTAL, COL_TT_SECONDS]:
    if c and c in eng_df:
        eng_df[c] = pd.to_numeric(eng_df[c], errors="coerce")

# Derive APV/AVD if missing
if COL_APV is None and COL_ACTIONS_TOTAL and COL_VISITS:
    eng_df["__apv_fallback"] = eng_df[COL_ACTIONS_TOTAL] / eng_df[COL_VISITS]
    COL_APV = "__apv_fallback"
if COL_AVD is None and COL_TT_SECONDS and COL_VISITS:
    eng_df["__avd_fallback"] = eng_df[COL_TT_SECONDS] / eng_df[COL_VISITS]
    COL_AVD = "__avd_fallback"

# ---- Reuse pre-repaired AVD if present; else compute MAD repair once ----
def mad_repair(series: pd.Series, k=MAD_K, replace_pct=REPLACE_WITH_PCT):
    s = series.astype(float).copy()
    med = np.nanmedian(s); mad = np.nanmedian(np.abs(s - med))
    sigma = 1.4826 * mad if mad > 0 else 0.0
    upper = med + k * sigma if sigma > 0 else np.inf
    p_lo  = np.nanpercentile(s, replace_pct) if s.notna().any() else med
    repaired = s.copy()
    flagged  = s > upper
    repaired[flagged] = p_lo
    return repaired, flagged

if "AvgTimePerVisit_sec_repair" in eng_df.columns:
    # Reuse existing repaired values and (optionally) the raw AVD
    if "AvgTimePerVisit_sec_raw" not in eng_df.columns:
        eng_df["AvgTimePerVisit_sec_raw"] = eng_df[COL_AVD] if COL_AVD else np.nan
    eng_df["AvgTimePerVisit_sec_repair"] = pd.to_numeric(eng_df["AvgTimePerVisit_sec_repair"], errors="coerce")
    if "flag_avd_inflated" not in eng_df.columns:
        eng_df["flag_avd_inflated"] = False
else:
    # No repaired column yet → do it once now
    if COL_AVD:
        eng_df["AvgTimePerVisit_sec_raw"] = eng_df[COL_AVD]
        repaired, flagged = mad_repair(eng_df[COL_AVD])
        eng_df["AvgTimePerVisit_sec_repair"] = repaired
        eng_df["flag_avd_inflated"] = flagged.fillna(False)
    else:
        eng_df["AvgTimePerVisit_sec_raw"] = np.nan
        eng_df["AvgTimePerVisit_sec_repair"] = np.nan
        eng_df["flag_avd_inflated"] = False

# ---- Robust scaling (0..1) ----
def robust_minmax(s, lo=ROBUST_LO, hi=ROBUST_HI):
    if s is None or s.isna().all():
        return pd.Series(np.zeros(len(eng_df)), index=eng_df.index)
    a = np.nanpercentile(s, lo); b = np.nanpercentile(s, hi)
    if not np.isfinite(a) or not np.isfinite(b) or b == a:
        return pd.Series(np.zeros(len(eng_df)), index=eng_df.index)
    return (s.clip(a, b) - a) / (b - a)

apv = eng_df[COL_APV] if COL_APV else pd.Series(np.nan, index=eng_df.index)
avd = eng_df["AvgTimePerVisit_sec_repair"]
vis = eng_df[COL_VISITS] if COL_VISITS else pd.Series(np.nan, index=eng_df.index)

eng_df["apv_scaled"] = robust_minmax(apv)          # NEW (scaled actions/visit)
eng_df["avd_scaled"] = robust_minmax(avd)          # NEW (scaled repaired time/visit)
eng_df["vis_scaled"] = robust_minmax(vis)          # NEW (scaled visits)

# ---- EngagementScore (weights auto-normalized) ----
ws = W_ACTIONS + W_TIME + W_VISITS
wa, wt, wv = W_ACTIONS/ws, W_TIME/ws, W_VISITS/ws
eng_df["EngagementScore"] = wa*eng_df["apv_scaled"] + wt*eng_df["avd_scaled"] + wv*eng_df["vis_scaled"]  # NEW

# Output
out_cols = [c for c in [COL_USER, COL_WEEK] if c] + [
    "EngagementScore",
    COL_APV if COL_APV else None,
    "AvgTimePerVisit_sec_raw","AvgTimePerVisit_sec_repair","flag_avd_inflated",
    COL_VISITS if COL_VISITS else None,
    "apv_scaled","avd_scaled","vis_scaled"
]
out_cols = [c for c in out_cols if c in eng_df.columns and c is not None]

eng_tidy = eng_df[out_cols].copy().sort_values("EngagementScore", ascending=False)
display(eng_tidy.head(20))


,UserId,Week,EngagementScore,Avg. Actions In Visit per Visit,AvgTimePerVisit_sec_raw,AvgTimePerVisit_sec_repair,flag_avd_inflated,Visits,apv_scaled,avd_scaled,vis_scaled
22246,437656,3,1.0,26.40,988.02,988.02,False,45,1.0,1.000000,1.0
21946,437377,1,1.0,25.02,1003.85,1003.85,False,47,1.0,1.000000,1.0
46584,499765,1,1.0,27.72,1088.35,1088.35,False,43,1.0,1.000000,1.0
67923,552961,7,1.0,26.46,972.97,972.97,False,63,1.0,1.000000,1.0
46686,499848,5,1.0,24.07,1219.93,1219.93,False,43,1.0,1.000000,1.0
46587,499765,4,1.0,24.21,1164.16,1164.16,False,43,1.0,1.000000,1.0
49817,507625,1,1.0,25.53,1206.98,1206.98,False,85,1.0,1.000000,1.0
21952,437377,7,1.0,23.83,901.67,901.67,False,48,1.0,1.000000,1.0
2846,10087,5,1.0,30.61,908.04,908.04,False,56,1.0,1.000000,1.0
27421,460604,5,1.0,24.16,1205.02,1205.02,False,45,1.0,1.000000,1.0


In [ ]:
# === Benchmark EngagementScore: percentiles, deciles, robust z, bands ===
import numpy as np
import pandas as pd
from IPython.display import display

dfb = eng_df.copy()

# 0) Guard: ensure EngagementScore exists
if "EngagementScore" not in dfb.columns:
    raise KeyError("EngagementScore not found. Run the scoring cell first.")

# 1) Calibration cohort = rows with a valid score
cal = dfb["EngagementScore"].astype(float)
cal_valid = cal[cal.notna()]
if cal_valid.empty:
    raise ValueError("No valid EngagementScore values to benchmark.")

# 2) Global percentiles & deciles
dfb["EngagementPercentile"] = cal.rank(pct=True) * 100  # 0..100
# Deciles: 10 (top) .. 1 (bottom)
dfb["EngagementDecile"] = pd.qcut(
    cal.rank(method="first"), 10, labels=[1,2,3,4,5,6,7,8,9,10]
).astype("Int64")

# 3) Within-week percentile (only if Week exists)
if "Week" in dfb.columns:
    dfb["WeekPercentile"] = (
        dfb.groupby("Week")["EngagementScore"].rank(pct=True) * 100
    )

# 4) Robust Z-score (median/MAD) — great for outlier sense-making
med = np.nanmedian(cal_valid)
mad = np.nanmedian(np.abs(cal_valid - med))
sigma = (1.4826 * mad) if mad > 0 else np.nan
if np.isfinite(sigma) and sigma > 0:
    dfb["EngagementZ_robust"] = (dfb["EngagementScore"] - med) / sigma
else:
    dfb["EngagementZ_robust"] = np.nan  # not enough spread

# 5) Bands from global percentiles (neutral wording)
# You can tune these cut points if you like.
p20 = cal_valid.quantile(0.20)
p50 = cal_valid.quantile(0.50)
p80 = cal_valid.quantile(0.80)

def depth_band(x):
    if pd.isna(x):          return "No Depth Data"
    if x >= p80:            return "High Depth"
    if x >= p50:            return "Good Depth"
    if x >= p20:            return "Light Depth"
    return "Quick Scan"  # scanners are valuable if cadence is high

dfb["DepthBand"] = dfb["EngagementScore"].map(depth_band)

# 6) Small calibration printout so you can benchmark meaningfully
print("=== EngagementScore calibration (this dataset) ===")
print(f"n = {cal_valid.size}")
print(f"min / p20 / p50 / p80 / max = "
      f"{cal_valid.min():.3f} / {p20:.3f} / {p50:.3f} / {p80:.3f} / {cal_valid.max():.3f}")
if np.isfinite(sigma) and sigma > 0:
    print(f"median = {med:.3f}, robust_sigma = {sigma:.3f}")


# 7) Nice preview (with colored benchmark columns)
cols_show = [c for c in [
    "UserId","Week","EngagementScore",
    "EngagementPercentile","EngagementDecile","EngagementZ_robust","DepthBand",
    "apv_scaled","avd_scaled","vis_scaled",
    "AvgTimePerVisit_sec_repair","flag_avd_inflated"
] if c in dfb.columns]

# assign a distinct background color per benchmark column
_col_colors = {
    "EngagementPercentile": "#e0f3ff",  # light blue
    "EngagementDecile":     "#e7ffe0",  # light green
    "EngagementZ_robust":   "#ffe9e9",  # light red/pink
    "DepthBand":            "#fff7d6",  # light yellow
}

styled = dfb[cols_show].head(20).style
for _col, _color in _col_colors.items():
    if _col in dfb.columns:
        styled = styled.set_properties(subset=[_col], **{"background-color": _color})

display(styled)


# Optional: export
out_bench = "/content/cleaned/engagement_benchmarked.xlsx"
dfb.to_excel(out_bench, index=False)
print("Saved:", out_bench)


=== EngagementScore calibration (this dataset) ===
n = 79984
min / p20 / p50 / p80 / max = 0.000 / 0.096 / 0.249 / 0.493 / 1.000
median = 0.249, robust_sigma = 0.222


,UserId,Week,EngagementScore,EngagementPercentile,EngagementDecile,EngagementZ_robust,DepthBand,apv_scaled,avd_scaled,vis_scaled,AvgTimePerVisit_sec_repair,flag_avd_inflated
0,0,1,0.448001,75.973945,8,0.896132,Good Depth,0.417824,0.420877,1.000000,370.890000,False
1,0,2,0.244333,49.127325,5,-0.022900,Light Depth,0.192566,0.225123,1.000000,199.260000,False
2,0,3,0.110052,23.070864,3,-0.628828,Light Depth,0.069861,0.051816,1.000000,47.310000,False
3,0,4,0.588867,86.824865,9,1.531773,High Depth,0.625168,0.481840,0.902439,424.340000,False
4,0,5,0.777932,97.094419,10,2.384911,High Depth,0.890730,0.580715,0.804878,511.030000,False
5,0,6,0.471001,78.084367,8,0.999915,Good Depth,0.477833,0.383718,1.000000,338.310000,False
6,0,7,0.522236,82.307712,9,1.231108,High Depth,0.523511,0.451797,1.000000,398.000000,False
7,0,8,0.162454,33.929286,4,-0.392371,Light Depth,0.136140,0.087914,1.000000,78.960000,False
8,1,1,0.323114,61.324140,7,0.332591,Good Depth,0.484998,0.070851,0.146341,64.000000,True
9,1,2,0.316566,60.350820,7,0.303045,Good Depth,0.359606,0.256648,0.219512,226.900000,False


Saved: /content/cleaned/engagement_benchmarked.xlsx


In [ ]:
# === Cadence labeling (WeeksActive + visits + days-since metrics) ===
import numpy as np
import pandas as pd
from IPython.display import display
from google.colab import files
from datetime import date

# Work on a copy, as requested
dfb = eng_df.copy()

# Column handles (soft detection)
def pick(cols, opts):
    for o in opts:
        if o in cols: return o
    return None

COL_USER = pick(dfb.columns, ["UserId"])
COL_WEEK = pick(dfb.columns, ["Week"])
COL_WA   = pick(dfb.columns, ["WeeksActive", "WeeksActive_eng", "WeeksActive_cad"])
COL_VIS  = pick(dfb.columns, ["Visits","Visits_eng","Visits_cad"])
COL_TOT  = pick(dfb.columns, ["Total Days since last visit"])
COL_MAX  = pick(dfb.columns, ["Max Days since last visit"])
COL_AVG  = pick(dfb.columns, ["Avg. Days since last visit per Visit"])

# Coerce numerics where relevant
for c in [COL_WA, COL_VIS, COL_TOT, COL_MAX, COL_AVG]:
    if c and c in dfb:
        dfb[c] = pd.to_numeric(dfb[c], errors="coerce")

def cadence_label(row):
    wa  = row.get(COL_WA, np.nan)
    vis = row.get(COL_VIS, np.nan)
    tot = row.get(COL_TOT, np.nan)   # Total Days since last visit
    mx  = row.get(COL_MAX, np.nan)   # Max Days since last visit
    avg = row.get(COL_AVG, np.nan)   # Avg. Days since last visit per Visit

    # Step 0: no data
    if pd.isna(wa) and pd.isna(vis) and pd.isna(tot) and pd.isna(avg):
        return "No Cadence Data"

    # Step 1: Check WeeksActive
    if pd.notna(wa):
        if wa == 0:
            return "Inactive"
        if wa == 1 and (pd.isna(vis) or vis <= 1):
            # Step 2 (Visits & recency) will refine "One-Time" vs "New"
            # Keep as a candidate; handled below with visits/recency
            pass
        if 2 <= wa <= 5:
            # Candidate for Sporadic / Biweekly; refined below by avg days
            pass
        if 6 <= wa:
            # Candidate for Weekly / Daily; refined below by avg days
            pass

    # Step 2: Visits & first-visit timing (proxy via Total Days since last visit)
    if pd.notna(vis) and vis == 1:
        if pd.notna(tot) and tot <= 14:
            return "New User"
        # 1 visit, older than 2 weeks
        return "One-Time Visitor"

    if pd.notna(vis) and vis > 1 and pd.notna(wa) and wa == 1:
        return "Binge Visitor"  # heavy in a single week then gone

    # Step 3: Avg. Days since last visit → base frequency class
    base = None
    if pd.notna(avg):
        if avg <= 1.5:
            base = "Daily Visitor"
        elif 5 <= avg <= 9:
            base = "Weekly Visitor"
        elif 10 <= avg <= 16:
            base = "Biweekly Visitor"
        elif avg > 16:
            base = "Sporadic Visitor"

    # Use WeeksActive to nudge unclear cases
    if base is None and pd.notna(wa):
        if wa >= 6:
            base = "Weekly Visitor"
        elif 2 <= wa <= 5:
            base = "Biweekly Visitor"
        elif wa == 1:
            base = "Sporadic Visitor"

    # Step 4: Validate with Max Days since last visit
    if pd.notna(mx) and mx > 20 and pd.notna(vis) and vis <= 2:
        # lean to One-Time/Sporadic if long gaps & low volume
        if pd.notna(vis) and vis == 1:
            return "One-Time Visitor"
        return "Sporadic Visitor"

    return base or "Sporadic Visitor"

def cadence_reason(row, label):
    bits = []
    wa  = row.get(COL_WA, np.nan)
    vis = row.get(COL_VIS, np.nan)
    tot = row.get(COL_TOT, np.nan)
    mx  = row.get(COL_MAX, np.nan)
    avg = row.get(COL_AVG, np.nan)

    if pd.notna(wa):  bits.append(f"WeeksActive={int(wa)}")
    if pd.notna(vis): bits.append(f"Visits={int(vis)}")
    if pd.notna(avg): bits.append(f"AvgDays≈{float(avg):.1f}")
    if pd.notna(tot): bits.append(f"TotalDaysSinceLast={int(tot)}")
    if pd.notna(mx):  bits.append(f"MaxDaysSinceLast={int(mx)}")

    # short rationale per label
    if label == "New User":
        bits.append("1 visit and recent (≤14d)")
    elif label == "One-Time Visitor":
        bits.append("1 visit, older than 2 weeks")
    elif label == "Binge Visitor":
        bits.append("many visits in a single week")
    elif label == "Daily Visitor":
        bits.append("avg ≤1.5 days between visits")
    elif label == "Weekly Visitor":
        bits.append("avg ~5–9 days between visits")
    elif label == "Biweekly Visitor":
        bits.append("avg ~10–16 days between visits")
    elif label == "Sporadic Visitor":
        bits.append(">16 days between visits or low activity")
    elif label == "Inactive":
        bits.append("no visits in window")

    return " | ".join(bits)

# Build cadence-only table
cad_cols = [c for c in [COL_USER, COL_WEEK, COL_WA, COL_VIS, COL_TOT, COL_MAX, COL_AVG] if c]
cadence_only = dfb[cad_cols].copy()

cadence_only["CadenceLabel"] = cadence_only.apply(cadence_label, axis=1)
cadence_only["CadenceWhy"]   = cadence_only.apply(lambda r: cadence_reason(r, r["CadenceLabel"]), axis=1)

# Export cadence-only
cad_out = f"/content/cleaned/cadence_labeled_{date.today().isoformat()}.xlsx"
cadence_only.to_excel(cad_out, index=False)
files.download(cad_out)

# Preview
display(cadence_only.head(20))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,UserId,Week,WeeksActive,Visits,Total Days since last visit,Max Days since last visit,Avg. Days since last visit per Visit,CadenceLabel,CadenceWhy
0,0,1,8,46,191,180,4.15,Weekly Visitor,WeeksActive=8 | Visits=46 | AvgDays≈4.2 | Tota...
1,0,2,8,224,271,64,1.21,Daily Visitor,WeeksActive=8 | Visits=224 | AvgDays≈1.2 | Tot...
2,0,3,8,652,346,319,0.53,Daily Visitor,WeeksActive=8 | Visits=652 | AvgDays≈0.5 | Tot...
3,0,4,8,38,184,71,4.84,Weekly Visitor,WeeksActive=8 | Visits=38 | AvgDays≈4.8 | Tota...
4,0,5,8,34,70,25,2.06,Weekly Visitor,WeeksActive=8 | Visits=34 | AvgDays≈2.1 | Tota...
5,0,6,8,45,230,56,5.11,Weekly Visitor,WeeksActive=8 | Visits=45 | AvgDays≈5.1 | Tota...
6,0,7,8,58,622,380,10.72,Biweekly Visitor,WeeksActive=8 | Visits=58 | AvgDays≈10.7 | Tot...
7,0,8,8,238,80,43,0.34,Daily Visitor,WeeksActive=8 | Visits=238 | AvgDays≈0.3 | Tot...
8,1,1,8,7,4,4,0.57,Daily Visitor,WeeksActive=8 | Visits=7 | AvgDays≈0.6 | Total...
9,1,2,8,10,4,2,0.40,Daily Visitor,WeeksActive=8 | Visits=10 | AvgDays≈0.4 | Tota...


In [ ]:
##New Fixed Logic
# === Cadence labeling (WeeksActive + visits + days-since metrics) ===
import numpy as np
import pandas as pd
from IPython.display import display
from google.colab import files
from datetime import date

# Work on a copy, as requested
dfb = eng_df.copy()

# Column handles (soft detection)
def pick(cols, opts):
    for o in opts:
        if o in cols: return o
    return None

COL_USER = pick(dfb.columns, ["UserId"])
COL_WEEK = pick(dfb.columns, ["Week"])
COL_WA   = pick(dfb.columns, ["WeeksActive", "WeeksActive_eng", "WeeksActive_cad"])
COL_VIS  = pick(dfb.columns, ["Visits","Visits_eng","Visits_cad"])
COL_TOT  = pick(dfb.columns, ["Total Days since last visit"])           # recency proxy for single-visit users
COL_MAX  = pick(dfb.columns, ["Max Days since last visit"])             # longest gap
COL_AVG  = pick(dfb.columns, ["Avg. Days since last visit per Visit"])  # average gap BETWEEN visits

# Coerce numerics where relevant
for c in [COL_WA, COL_VIS, COL_TOT, COL_MAX, COL_AVG]:
    if c and c in dfb:
        dfb[c] = pd.to_numeric(dfb[c], errors="coerce")

def cadence_label(row):
    wa  = row.get(COL_WA,  np.nan)
    vis = row.get(COL_VIS, np.nan)
    tot = row.get(COL_TOT, np.nan)  # Total Days since last visit (used for single-visit recency)
    mx  = row.get(COL_MAX, np.nan)  # Max Days since last visit (sanity check)
    avg = row.get(COL_AVG, np.nan)  # Avg. Days since last visit per Visit (MEANINGFUL ONLY IF vis >= 2)

    # Step 0: no data at all
    if pd.isna(wa) and pd.isna(vis) and pd.isna(tot) and pd.isna(avg):
        return "No Cadence Data"

    # Step 1: WeeksActive hard check
    if pd.notna(wa):
        if wa == 0:
            return "Inactive"
        # wa == 1 remains unresolved here; refined below by visits/recency

    # Step 2: Handle single-visit users FIRST (so they never become "Daily" by accident)
    if pd.notna(vis) and vis == 1:
        # If recent enough, call them New; otherwise One-Time
        if pd.notna(tot) and tot <= 14:
            return "New User"
        return "One-Time Visitor"

    # Step 2b: Binge pattern — multiple visits but only in a single week
    if pd.notna(vis) and vis > 1 and pd.notna(wa) and wa == 1:
        return "Binge Visitor"

    # Step 3: Base frequency using avg gap BETWEEN visits — ONLY when vis >= 2
    base = None
    if pd.notna(avg) and pd.notna(vis) and vis >= 2:
        # Guard weird zeros caused by same-day logging; treat as ultra-frequent
        if avg <= 1.5:
            base = "Daily Visitor"
        elif 5 <= avg <= 9:
            base = "Weekly Visitor"
        elif 10 <= avg <= 16:
            base = "Biweekly Visitor"
        elif avg > 16:
            base = "Sporadic Visitor"

    # Step 3b: If avg is missing or vis < 2 (already filtered above), fall back to WeeksActive
    if base is None and pd.notna(wa):
        if wa >= 6:
            base = "Weekly Visitor"
        elif 2 <= wa <= 5:
            base = "Biweekly Visitor"
        elif wa == 1:
            base = "Sporadic Visitor"

    # Step 4: Sanity check with Max gap for low-visit users (nudge toward low-engagement)
    if pd.notna(mx) and mx > 20 and pd.notna(vis) and vis <= 2:
        if vis == 1:
            return "One-Time Visitor"
        return "Sporadic Visitor"

    return base or "Sporadic Visitor"

def cadence_reason(row, label):
    bits = []
    wa  = row.get(COL_WA,  np.nan)
    vis = row.get(COL_VIS, np.nan)
    tot = row.get(COL_TOT, np.nan)
    mx  = row.get(COL_MAX, np.nan)
    avg = row.get(COL_AVG, np.nan)

    if pd.notna(wa):  bits.append(f"WeeksActive={int(wa)}")
    if pd.notna(vis): bits.append(f"Visits={int(vis)}")
    if pd.notna(avg): bits.append(f"AvgDaysBetween≈{float(avg):.1f}")  # clarify meaning
    if pd.notna(tot): bits.append(f"TotalDaysSinceLast={int(tot)}")
    if pd.notna(mx):  bits.append(f"MaxDaysSinceLast={int(mx)}")

    # short rationale per label
    if label == "New User":
        bits.append("1 visit and recent (≤14d)")
    elif label == "One-Time Visitor":
        bits.append("1 visit, older than 2 weeks")
    elif label == "Binge Visitor":
        bits.append("many visits concentrated in a single week")
    elif label == "Daily Visitor":
        bits.append("avg gap ≤1.5 days (vis≥2)")
    elif label == "Weekly Visitor":
        bits.append("avg gap ~5–9 days (or WA≥6)")
    elif label == "Biweekly Visitor":
        bits.append("avg gap ~10–16 days (or 2–5 active weeks)")
    elif label == "Sporadic Visitor":
        bits.append(">16-day gaps or low activity")
    elif label == "Inactive":
        bits.append("no visits in window")

    return " | ".join(bits)

# Build cadence-only table
cad_cols = [c for c in [COL_USER, COL_WEEK, COL_WA, COL_VIS, COL_TOT, COL_MAX, COL_AVG] if c]
cadence_only = dfb[cad_cols].copy()

cadence_only["CadenceLabel"] = cadence_only.apply(cadence_label, axis=1)
cadence_only["CadenceWhy"]   = cadence_only.apply(lambda r: cadence_reason(r, r["CadenceLabel"]), axis=1)

# Export cadence-only
cad_out = f"/content/cleaned/cadence_labeled_{date.today().isoformat()}.xlsx"
cadence_only.to_excel(cad_out, index=False)
files.download(cad_out)

# Preview
display(cadence_only.head(20))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,UserId,Week,WeeksActive,Visits,Total Days since last visit,Max Days since last visit,Avg. Days since last visit per Visit,CadenceLabel,CadenceWhy
0,0,1,8,46,113.0,113.0,18.83,Sporadic Visitor,WeeksActive=8 | Visits=46 | AvgDaysBetween≈18....
1,0,2,8,224,NaN,NaN,NaN,Weekly Visitor,WeeksActive=8 | Visits=224 | avg gap ~5–9 days...
2,0,3,8,652,NaN,NaN,NaN,Weekly Visitor,WeeksActive=8 | Visits=652 | avg gap ~5–9 days...
3,0,4,8,38,NaN,NaN,NaN,Weekly Visitor,WeeksActive=8 | Visits=38 | avg gap ~5–9 days ...
4,0,5,8,34,NaN,NaN,NaN,Weekly Visitor,WeeksActive=8 | Visits=34 | avg gap ~5–9 days ...
5,0,6,8,45,NaN,NaN,NaN,Weekly Visitor,WeeksActive=8 | Visits=45 | avg gap ~5–9 days ...
6,0,7,8,58,NaN,NaN,NaN,Weekly Visitor,WeeksActive=8 | Visits=58 | avg gap ~5–9 days ...
7,0,8,8,238,NaN,NaN,NaN,Weekly Visitor,WeeksActive=8 | Visits=238 | avg gap ~5–9 days...
8,1,1,8,7,1.0,1.0,1.00,Daily Visitor,WeeksActive=8 | Visits=7 | AvgDaysBetween≈1.0 ...
9,1,2,8,10,NaN,NaN,NaN,Weekly Visitor,WeeksActive=8 | Visits=10 | avg gap ~5–9 days ...


In [ ]:
# === Side-by-side: DepthBand (engagement) + CadenceLabel (cadence) — minimal view ===
import numpy as np
import pandas as pd
from IPython.display import display

# 0) Base frames
dfb = eng_df.copy()

if "EngagementScore" not in dfb.columns:
    raise KeyError("EngagementScore missing. Run the scoring/benchmark cell first.")

# 1) Ensure engagement highlights exist (create on-the-fly if absent)
cal = dfb["EngagementScore"].astype(float)
valid = cal.dropna()
p20 = valid.quantile(0.20) if not valid.empty else np.nan
p50 = valid.quantile(0.50) if not valid.empty else np.nan
p80 = valid.quantile(0.80) if not valid.empty else np.nan

if "DepthBand" not in dfb.columns:
    def _depth_band(x):
        if pd.isna(x): return "No Depth Data"
        if x >= p80:   return "High Depth"
        if x >= p50:   return "Good Depth"
        if x >= p20:   return "Light Depth"
        return "Quick Scan"
    dfb["DepthBand"] = dfb["EngagementScore"].map(_depth_band)

if "EngagementPercentile" not in dfb.columns:
    dfb["EngagementPercentile"] = cal.rank(pct=True) * 100

if "EngagementDecile" not in dfb.columns:
    dfb["EngagementDecile"] = pd.qcut(
        cal.rank(method="first"),
        10, labels=[1,2,3,4,5,6,7,8,9,10]
    ).astype("Int64")

# 2) Minimal engagement columns (keys + highlights)
eng_cols = [c for c in ["UserId","Week","DepthBand","EngagementPercentile","EngagementDecile"] if c in dfb.columns]

# 3) Minimal cadence columns
if "cadence_only" not in globals():
    raise NameError("cadence_only not found. Run the cadence labeling cell first.")

cad_cols = [c for c in ["UserId","Week","CadenceLabel"] if c in cadence_only.columns]

# 4) Join on available keys
keys = [k for k in ["UserId","Week"] if k in dfb.columns and k in cadence_only.columns]
side_by_side = dfb[eng_cols].merge(cadence_only[cad_cols], on=keys, how="left")

# 5) Styled preview — highlight just the two labels
_col_colors = {"DepthBand": "#fff7d6", "CadenceLabel": "#e7ffe0"}  # yellow / green
styled = side_by_side.head(30).style
for _col, _color in _col_colors.items():
    if _col in side_by_side.columns:
        styled = styled.set_properties(subset=[_col], **{"background-color": _color})

display(styled)

# Optional export of this tight view
out_path = "/content/cleaned/engagement_depthband__cadence_label.xlsx"
side_by_side.to_excel(out_path, index=False)
print("Saved:", out_path)


,UserId,Week,DepthBand,EngagementPercentile,EngagementDecile,CadenceLabel
0,0,1,Good Depth,75.973945,8,Sporadic Visitor
1,0,2,Light Depth,49.127325,5,Weekly Visitor
2,0,3,Light Depth,23.070864,3,Weekly Visitor
3,0,4,High Depth,86.824865,9,Weekly Visitor
4,0,5,High Depth,97.094419,10,Weekly Visitor
5,0,6,Good Depth,78.084367,8,Weekly Visitor
6,0,7,High Depth,82.307712,9,Weekly Visitor
7,0,8,Light Depth,33.929286,4,Weekly Visitor
8,1,1,Good Depth,61.324140,7,Daily Visitor
9,1,2,Good Depth,60.350820,7,Weekly Visitor


Saved: /content/cleaned/engagement_depthband__cadence_label.xlsx
